In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [41]:
import requests
import pandas as pd
from datetime import datetime, timedelta, date

base_url = 'https://api.energy-charts.info/'
country_code = 'ch'

def process_timestamps(unix_seconds):
    return [datetime.utcfromtimestamp(ts) for ts in unix_seconds]

def get_public_power():
    start_date = date(2010, 1, 1)
    end_date = date(2024, 4, 23)

    start_datetime = start_date.strftime('%Y-%m-%dT%H:%M:%S%z')
    end_datetime = end_date.strftime('%Y-%m-%dT%H:%M:%S%z')
    
    url = base_url + 'public_power'
    params = {
        'country': country_code,
        'start': start_datetime,
        'end': end_datetime
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Failed to retrieve data: {response.status_code} - {response.text}")
        return pd.DataFrame()

    data = response.json()
    timestamps = process_timestamps(data['unix_seconds'])
    production_data = data['production_types']

    df_list = []
    for entry in production_data:
        if entry['name'] == 'Hydro Run-of-River':
            df = pd.DataFrame({
                'Time': timestamps,
                'Production Type': entry['name'],
                'Power (MW)': entry['data']
            })
            df_list.append(df)

    if not df_list:
        return pd.DataFrame()

    return pd.concat(df_list, ignore_index=True)

# Load data
df = get_public_power()

df['Time'] = pd.to_datetime(df['Time'])
df.set_index('Time', inplace=True)
    

data = df['Power (MW)'].values.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

print(df.head(25))


In [42]:
df = df.dropna(how='any')
print(df.head(25))


In [43]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def create_dataset(data,
 look_back=1):
    """
    Convert an array of values into a dataset matrix suitable for LSTM training.
    
    Parameters:
        data (array): The dataset array containing sequential data.
        look_back (int): Number of previous time steps to use as input variables to predict the next time period.
        
    Returns:
        X (array): Feature dataset.
        y (array): Labels.
    """
    X, y = [], []
    for i in range(len(data) - look_back):
        a = data[i:(i + look_back), 0]  # Fetch the input data from the current index to the look_back length
        X.append(a)
        y.append(data[i + look_back, 0])  # The label will be the next data point
    return np.array(X), np.array(y)

# Assume 'data' is loaded and scaled as the 'Power (MW)' from your solar data.
# For example purposes, let's define 'data' here manually:
data = np.array([[i] for i in range(100)])  # Example data generation
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

# Define the look_back period
look_back = 24  # This means the input for each sample will be sequences of 3 past observations

# Create the dataset
X, y = create_dataset(data, look_back)

# Reshape input to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

print("Sample of feature (X) and label (y):")
print("X[0]:", X[0])
print("y[0]:", y[0])

In [63]:
model = Sequential([
     LSTM(60, input_shape=(look_back, 1)),
     Dense(1)
])

optimizer = Adam(learning_rate=0.01) 
model.compile(optimizer=optimizer, loss='mean_squared_error')
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#callbacks=[early_stopping]
history = model.fit(X, y, epochs=200, batch_size=16, validation_split=0.2, verbose=1)

model.summary()


In [64]:

# Plot training loss and validation loss
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()


In [65]:
predictions = model.predict(X)

predictions = scaler.inverse_transform(predictions)
actuals = scaler.inverse_transform([y])

rmse = np.sqrt(mean_squared_error(actuals[0], predictions[:,0]))
mape = np.mean(np.abs((actuals[0] - predictions[:,0]) / actuals[0])) * 100
r_squared = 1 - (np.sum((actuals[0] - predictions[:,0])**2) / np.sum((actuals[0] - np.mean(actuals[0]))**2))
print(f'RMSE: {rmse}')
print(f'MAPE: {mape}')
print(f'R^2: {r_squared}')

model.save('hydro_run_of_river_lstm.keras')

In [66]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('solar_power_lstm.keras')  # Update the path to where your model is saved

# Scale and prepare the last 24 hours of data
recent_data_scaled = scaler.transform(df['Power (MW)'].tail(24).values.reshape(-1, 1))
recent_data_scaled = recent_data_scaled.reshape(1, 24, 1)

predictions = []
current_batch = recent_data_scaled

# Predict iteratively
for i in range(24):  # Predict the next 24 hours
    current_pred = model.predict(current_batch)[0]  # Predict the next hour
    predictions.append(current_pred)  # Store the prediction
    current_batch = np.append(current_batch[:,1:,:], [[current_pred]], axis=1)  # Update the batch to include the new prediction

# Inverse scale predictions
predicted_original = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

# Print or store the predictions
predicted_times = pd.date_range(start=df.index[-1] + pd.Timedelta(hours=1), periods=24, freq='H')
predicted_df = pd.DataFrame(data=predicted_original.flatten(), index=predicted_times, columns=['Predicted Power (MW)'])
print(predicted_df)
